# Data Exploration Analysis For the Sales Prediction Data
### EDA Steps:
1. Data Assessing 
2. Data Cleaning
3. Univariate EDA
4. Bivariate EDA
5. Multivariate EDA
6. Outliers Detection

In [11]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor


# 1. Data Assessment

In [2]:
df = pd.read_csv(r"C:\Users\USER\Desktop\sales pred\Train.csv")
df.head()

,Product code,Product name,Unit name,Quantity,Rate,Amount,Tax,Gross amount,Month
0,12,7DAYS BABY WIPES,PIECE,1.0,8500,7203,1297,8500.0,January
1,27,ABC DENT 70G,PIECE,1.0,2000,1695,305,2000.0,January
2,61,AFROCARE JELLY 100G,PIECE,1.0,3000,2542,458,3000.0,January
3,62,AFROCARE PERFUMED GLYCERINE,PIECE,1.0,2500,2119,381,2500.0,January
4,66,AKABANGA CHILI OIL 20ML,PIECE,1.0,2500,2119,381,2500.0,January


In [3]:
df.shape

(16821, 9)

In [4]:
df.columns

Index(['Product code', 'Product name ', 'Unit name', 'Quantity', 'Rate',
       'Amount', 'Tax', 'Gross amount', 'Month'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16821 entries, 0 to 16820
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Product code   16821 non-null  int64  
 1   Product name   16821 non-null  object 
 2   Unit name      16821 non-null  object 
 3   Quantity       16821 non-null  float64
 4   Rate           16821 non-null  int64  
 5   Amount         16821 non-null  int64  
 6   Tax            16821 non-null  int64  
 7   Gross amount   16821 non-null  float64
 8   Month          16821 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 1.2+ MB


In [6]:
df.describe()

,Product code,Quantity,Rate,Amount,Tax,Gross amount
count,16821.000000,16821.000000,16821.000000,1.682100e+04,16821.000000,1.682100e+04
mean,4046.217764,22.846606,7139.076571,5.345158e+04,9541.528447,6.299311e+04
std,2303.901050,94.423055,8183.624863,1.620581e+05,28981.154461,1.908575e+05
min,3.000000,0.000000,100.000000,0.000000e+00,0.000000,0.000000e+00
25%,1977.000000,1.000000,2500.000000,7.627000e+03,1373.000000,9.000000e+03
50%,4074.000000,4.000000,4650.000000,1.778400e+04,3051.000000,2.098500e+04
75%,5816.000000,11.000000,8500.000000,4.067800e+04,7322.000000,4.800000e+04
max,8048.000000,2935.000000,150000.000000,3.071610e+06,552890.000000,3.624500e+06


In [7]:
df.isnull().sum()

Product code     0
Product name     0
Unit name        0
Quantity         0
Rate             0
Amount           0
Tax              0
Gross amount     0
Month            0
dtype: int64

In [8]:
df.duplicated().sum()

0

In [9]:
# define numerical & categorical columns in train data
numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']

# print numerical & categorical columns in train data
print('We have {} numerical features in train data and they as as follows : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features train data and they as as follows: {}'.format(len(categorical_features), categorical_features))

We have 6 numerical features in train data and they as as follows : ['Product code', 'Quantity', 'Rate', 'Amount', 'Tax', 'Gross amount']

We have 3 categorical features train data and they as as follows: ['Product name ', 'Unit name', 'Month']


In [13]:


# --- Data Cleaning ---
# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Standardize the month column (keep as string, just clean up formatting)
df['month'] = df['month'].astype(str).str.strip().str.title()

# Remove rows with zero quantity or gross_amount
df = df[df['quantity'] > 0]
df = df[df['gross_amount'] > 0]

# Round gross_amount to nearest integer to avoid small discrepancies
df['gross_amount'] = df['gross_amount'].round().astype(int)

# Standardize product_code and product_name mapping
code_name_mapping = df.groupby('product_code')['product_name'].nunique()
inconsistent_codes = code_name_mapping[code_name_mapping > 1]
for code in inconsistent_codes.index:
    most_frequent_name = df[df['product_code'] == code]['product_name'].mode()[0]
    df.loc[df['product_code'] == code, 'product_name'] = most_frequent_name

# Trim spaces and standardize strings
df['product_name'] = df['product_name'].str.strip()
df['unit_name'] = df['unit_name'].str.strip().str.title()

# Recalculate amount and tax
df['amount'] = (df['quantity'] * df['rate']).round().astype(int)
df['tax'] = (df['gross_amount'] - df['amount']).astype(int)

# Cap outliers using IQR for quantity, amount, gross_amount, and rate (skip tax)
for col in ['quantity', 'amount', 'gross_amount', 'rate']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound).astype(int)

# Save the cleaned dataset
df.to_csv('cleaned_train_dataset.csv', index=False)
print("Cleaned training dataset saved as 'cleaned_train_dataset.csv'")

# --- Data Assessment ---
print("\n### 1. Data Assessment")
print("\nShape of the dataset:", df.shape)
print("\nColumns in the dataset:", df.columns)
print("\nData types and info:")
df.info()
print("\nSummary statistics:")
print(df.describe())
print("\nMissing values in each column:")
print(df.isnull().sum())
print("\nNumber of duplicated rows:", df.duplicated().sum())

# Define numerical & categorical columns
numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print('\nWe have {} numerical features in train data: {}'.format(len(numeric_features), numeric_features))
print('We have {} categorical features in train data: {}'.format(len(categorical_features), categorical_features))

# Check unique values in categorical features
print('\nNumber of unique data points in categorical features in the train dataset:')
print('Number of unique data points in Product code:', df['product_code'].nunique(), df['product_code'].unique())
print('Number of unique data points in Product name:', df['product_name'].nunique(), df['product_name'].unique())
print('Number of unique data points in Unit name:', df['unit_name'].nunique(), df['unit_name'].unique())
print('Number of unique data points in Month:', df['month'].nunique(), df['month'].unique())

# --- Additional Cleaning Checks ---
# Verify one-to-one mapping between 'product_code' and 'product_name'
code_name_mapping = df.groupby('product_code')['product_name'].nunique()
inconsistent_codes = code_name_mapping[code_name_mapping > 1]
if not inconsistent_codes.empty:
    print("\nProduct codes with multiple names after fix:")
    print(inconsistent_codes)
else:
    print("\nAll product codes map to exactly one product name after fix.")

# Check for case inconsistencies or extra spaces in 'unit_name' and 'month'
print("\nUnique values in 'unit_name':", df['unit_name'].unique())
print("Unique values in 'month':", df['month'].unique())

# Validate numeric columns: Check for negative values
negative_quantity = df[df['quantity'] < 0]
negative_rate = df[df['rate'] < 0]
if not negative_quantity.empty:
    print("\nRows with negative quantity:")
    print(negative_quantity[['product_code', 'product_name', 'quantity']])
if not negative_rate.empty:
    print("\nRows with negative rate:")
    print(negative_rate[['product_code', 'product_name', 'rate']])

# --- Product Performance Analysis ---
print("\n### Product Performance")
# Set plot style
sns.set(style="whitegrid")

# Group by product_name for performance analysis
product_performance = df.groupby('product_name').agg({
    'gross_amount': 'sum',
    'quantity': 'sum',
    'tax': 'sum',
    'product_code': 'first'  # product_code for reference
}).reset_index()

# Sort by gross_amount to find top-performing products
top_products = product_performance.sort_values(by='gross_amount', ascending=False).head(10)
print("\nTop 10 Products by Total Gross Amount:")
print(top_products[['product_name', 'product_code', 'gross_amount', 'quantity']])

# --- Univariate EDA ---
print("\n### Univariate EDA")
# Distribution Plots for Numeric Columns
numeric_cols = ['quantity', 'rate', 'amount', 'tax', 'gross_amount']
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(3, 2, i)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
plt.tight_layout()
plt.savefig('train_distribution_plots.png')
plt.close()

# Count Plots for Categorical Columns
categorical_cols = ['unit_name', 'month']
plt.figure(figsize=(15, 5))
for i, col in enumerate(categorical_cols, 1):
    plt.subplot(1, 2, i)
    sns.countplot(data=df, x=col, order=df[col].value_counts().index)
    plt.title(f'Count Plot of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('train_count_plots.png')
plt.close()

# --- Bivariate EDA ---
print("\n### Bivariate EDA")
# Scatter Plot: Quantity vs. Gross Amount
plt.figure(figsize=(10, 6))
sns.scatterplot(data=product_performance, x='quantity', y='gross_amount', hue='gross_amount', size='gross_amount')
plt.title('Scatter Plot: Quantity vs. Gross Amount by Product')
plt.xlabel('Total Quantity Sold')
plt.ylabel('Total Gross Amount')
plt.savefig('train_scatter_quantity_vs_gross_amount.png')
plt.close()

# --- Multivariate EDA ---
print("\n### Multivariate EDA")
# Correlation Matrix
plt.figure(figsize=(10, 8))
corr_matrix = df[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Numeric Columns')
plt.savefig('train_correlation_matrix.png')
plt.close()

# Variance Inflation Factor (VIF) for Numeric Columns
vif_data = df[numeric_cols].dropna()
vif_results = pd.DataFrame()
vif_results['Feature'] = numeric_cols
vif_results['VIF'] = [variance_inflation_factor(vif_data.values, i) for i in range(vif_data.shape[1])]
print("\nVariance Inflation Factor (VIF) for Numeric Columns:")
print(vif_results)

# --- Outlier Detection ---
print("\n### Outlier Detection")
# Box Plots for Numeric Columns
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(3, 2, i)
    sns.boxplot(y=df[col])
    plt.title(f'Box Plot of {col}')
plt.tight_layout()
plt.savefig('train_box_plots.png')
plt.close()

# IQR-Based Outlier Detection
outliers_summary = {}
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outliers_summary[col] = {
        'num_outliers': len(outliers),
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }
print("\nOutlier Summary (IQR Method):")
for col, info in outliers_summary.items():
    print(f"{col}: {info['num_outliers']} outliers (lower: {info['lower_bound']:.2f}, upper: {info['upper_bound']:.2f})")

# Visualize Outliers in Gross Amount
gross_outliers = product_performance[
    (product_performance['gross_amount'] < outliers_summary['gross_amount']['lower_bound']) |
    (product_performance['gross_amount'] > outliers_summary['gross_amount']['upper_bound'])
]
plt.figure(figsize=(10, 6))
sns.scatterplot(data=product_performance, x='quantity', y='gross_amount', color='blue', label='Normal')
sns.scatterplot(data=gross_outliers, x='quantity', y='gross_amount', color='red', label='Outliers')
plt.title('Scatter Plot with Outliers in Gross Amount')
plt.xlabel('Total Quantity Sold')
plt.ylabel('Total Gross Amount')
plt.legend()
plt.savefig('train_outliers_gross_amount.png')
plt.close()

# --- Save Top Products for Stocking Recommendations ---
top_products.to_csv('train_top_products.csv', index=False)
print("\nTop products saved to 'train_top_products.csv' for stocking recommendations.")

Cleaned training dataset saved as 'cleaned_train_dataset.csv'

### 1. Data Assessment

Shape of the dataset: (16816, 9)

Columns in the dataset: Index(['product_code', 'product_name', 'unit_name', 'quantity', 'rate',
       'amount', 'tax', 'gross_amount', 'month'],
      dtype='object')

Data types and info:
<class 'pandas.core.frame.DataFrame'>
Index: 16816 entries, 0 to 16820
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_code  16816 non-null  int64 
 1   product_name  16816 non-null  object
 2   unit_name     16816 non-null  object
 3   quantity      16816 non-null  int32 
 4   rate          16816 non-null  int32 
 5   amount        16816 non-null  int32 
 6   tax           16816 non-null  int32 
 7   gross_amount  16816 non-null  int32 
 8   month         16816 non-null  object
dtypes: int32(5), int64(1), object(3)
memory usage: 985.3+ KB

Summary statistics:
       product_code      quantity        